### 代理ip爬取

In [ ]:
#爬取代理ip
import requests
import re
  
response = requests.get("https://www.sslproxies.org/")
proxy_ips = re.findall('\d+\.\d+\.\d+\.\d+:\d+', response.text)
 
valid_ips = []
for ip in proxy_ips:
  try:
    result = requests.get('https://ip.seeip.org/jsonip?',
			    proxies={'http': ip, 'https': ip},
			    timeout=5)
    print(result.json())
    valid_ips.append(ip)
  except:
      print(f"{ip} invalid")

176.57.188.32:443 invalid
133.242.237.138:3128 invalid
65.108.136.229:9090 invalid
20.212.168.165:3128 invalid
{'ip': '102.130.77.201'}
164.155.146.0:80 invalid
164.155.150.0:80 invalid
164.155.145.0:80 invalid
159.192.253.197:8080 invalid
164.155.149.1:80 invalid
164.155.147.31:80 invalid
185.92.36.61:443 invalid
20.47.108.204:8888 invalid
164.155.146.31:80 invalid
164.155.147.1:80 invalid
161.97.158.118:1081 invalid
213.230.97.10:3128 invalid
164.155.148.1:80 invalid
85.195.104.71:80 invalid
5.189.184.6:80 invalid
145.239.169.47:1080 invalid
181.196.205.250:38178 invalid
37.26.86.206:47464 invalid
192.155.107.214:1080 invalid
61.54.220.36:1080 invalid
151.106.13.218:1080 invalid
151.106.18.123:1080 invalid
44.233.12.234:1080 invalid
154.236.179.235:1976 invalid
185.237.99.218:443 invalid
164.155.148.0:80 invalid
164.155.148.31:80 invalid
164.155.150.31:80 invalid
164.155.146.1:80 invalid
59.98.82.114:808 invalid
164.155.145.31:80 invalid
151.106.17.124:1080 invalid
66.94.97.238:443 i

In [ ]:
#成功的ip
valid_ips

['102.130.79.1:3128',
 '167.114.96.27:9300',
 '45.167.125.209:9992',
 '45.190.84.2:999',
 '45.190.84.3:999',
 '157.100.12.138:999',
 '47.241.165.133:443',
 '190.97.226.236:999',
 '159.65.133.175:31280',
 '148.251.66.8:3128',
 '106.75.171.235:8080']

### Dcard基金爬蟲

In [ ]:
!pip install fake_useragent

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=01c7cc8bd3e61bb8fbc736169942a5f09881d8ad5bef3a25207296114d8a6d64
  Stored in directory: /root/.cache/pip/wheels/ed/f7/62/50ab6c9a0b5567267ab76a9daa9d06315704209b2c5d032031
Successfully built fake-useragent


In [ ]:
!pip install cloudscraper

     |████████████████████████████████| 97 kB 2.9 MB/s 
     |████████████████████████████████| 54 kB 2.7 MB/s 


In [ ]:
def dcard_api(url):
  import requests,json,random,cloudscraper
  from fake_useragent import UserAgent

  user_agent = UserAgent()
  
  ##填入代理ip
  proxy_list = []
  
  proxy = random.choice(proxy_list)
  headers={'user-agent': user_agent.random,
      'Referer':url}
  res = cloudscraper.create_scraper().get(url,headers=headers,proxies={'http':'http://'+proxy}).text
  resjson = json.loads(res)
  return resjson

In [ ]:
#爬取文章id
def Crawl(ID):
  from datetime import datetime
  import time,re
  global df

  url = 'https://www.dcard.tw/service/api/v2/posts/' + str(ID)
  resjson = dcard_api(url)
  if resjson!=[] and 'error' not in resjson:
    #時間格式轉換
    timeset=re.compile(r'\d\d\d\d-\d\d-\d\dT\d\d:\d\d:\d\d')
    createdtime=timeset.search(resjson['createdAt']).group().split("T")
    date=datetime.strptime(createdtime[0]+" "+createdtime[1], "%Y-%m-%d %H:%M:%S")
    if resjson['anonymousSchool']==False:
      author=resjson['school']
    else:
      author='匿名'
    data=[{'主文/留言':"主文",
          '作者':author,
          '時間':str(date),
          '標題':resjson['title'],
          '內容':resjson['content']}]
    df=df.append(data)
    print(data)
    time.sleep(5)
    
    #留言爬取
    if resjson['commentCount'] >0:
      Crawl_comment(ID,resjson['title'],resjson['commentCount'])

In [ ]:
#爬取留言
def Crawl_comment(ID,title,commentCount):
  from datetime import datetime
  import time,re
  global df

  pages = commentCount//30
  if commentCount%30==0:
    pages-=1
  for page in range(pages+1):
    url = 'https://www.dcard.tw/service/api/v2/posts/' + str(ID) + "/comments?" + "after=" + str(page*30)
    resjson = dcard_api(url)

    if resjson!=[] and 'error' not in resjson:
      for i in range(len(resjson)):
        if resjson[i]['hidden']==False:
          timeset=re.compile(r'\d\d\d\d-\d\d-\d\dT\d\d:\d\d:\d\d')
          createdtime=timeset.search(resjson[i]['createdAt']).group().split("T")
          date=datetime.strptime(createdtime[0]+" "+createdtime[1], "%Y-%m-%d %H:%M:%S")
          if resjson[i]['host']==False:
            author=resjson[i]['school']
          else:
            author="原Po"
          data=[{'主文/留言':"留言",
              '作者':author,
              '時間':str(date),
              '標題':title,
              '內容':resjson[i]['content']}]
          df=df.append(data)
          print(data)
    time.sleep(1)

In [ ]:
import pandas as pd
import requests,json,time,re
from datetime import datetime
from fake_useragent import UserAgent

user_agent = UserAgent()
df= pd.DataFrame()

##輸入欲爬之時間段
date_s=datetime.strptime("2021-08-20 00:00:00","%Y-%m-%d %H:%M:%S") 
date_f=datetime.strptime("2021-08-22 00:00:00","%Y-%m-%d %H:%M:%S")
timeset=re.compile(r'\d\d\d\d-\d\d-\d\dT\d\d:\d\d:\d\d')

url = "https://www.dcard.tw/service/api/v2/search/posts?query=%E5%9F%BA%E9%87%91&field=topics&sort=created&limit=1000"
resjson = dcard_api(url)
if resjson != []:
  for article in resjson:
    createdtime=timeset.search(article['createdAt']).group().split("T")
    date=datetime.strptime(createdtime[0]+" "+createdtime[1], "%Y-%m-%d %H:%M:%S")
    if date >= date_s and date < date_f and article['createdAt']!="":
      Crawl(article['id'])
      time.sleep(2)

[{'主文/留言': '主文', '作者': '國立臺南大學', '時間': '2021-08-20 02:18:51', '標題': '集保e存摺', '內容': '請問大家都是買哪支股票呢？我第一次接觸，若把錢先投入聯電、長榮、台積電等大公司會不會比較保險呢？謝謝回覆！'}]
[{'主文/留言': '留言', '作者': 'Chiang-chi', '時間': '2021-08-20 02:41:58', '標題': '集保e存摺', '內容': '0050應該會比他們3家各1/3還安全'}]
[{'主文/留言': '留言', '作者': '國立虎尾科技大學', '時間': '2021-08-20 03:02:08', '標題': '集保e存摺', '內容': '先爬文不要伸手比較保險'}]
[{'主文/留言': '留言', '作者': '淡江大學', '時間': '2021-08-20 03:55:54', '標題': '集保e存摺', '內容': '全部買劍湖山最保險，讓你資產從小累積'}]
[{'主文/留言': '留言', '作者': '臺灣警察專科學校', '時間': '2021-08-20 04:00:14', '標題': '集保e存摺', '內容': '長榮..\n你來亂的吧？'}]
[{'主文/留言': '留言', '作者': 'yomi', '時間': '2021-08-20 07:46:51', '標題': '集保e存摺', '內容': 'Allin 萬海 別墅靠海\nAllin 陽明 人生光明\nAllin 長榮 全家光榮'}]
[{'主文/留言': '留言', '作者': '國立虎尾科技大學', '時間': '2021-08-20 09:24:48', '標題': '集保e存摺', '內容': '不知道是不是反串😂\n認真回：你全部丟0050吧~'}]
[{'主文/留言': '留言', '作者': '長庚科技大學', '時間': '2021-08-20 09:40:55', '標題': '集保e存摺', '內容': '買開發金最安全'}]
[{'主文/留言': '留言', '作者': '國立彰化師範大學', '時間': '2021-08-20 10:23:20', '標題': '集保e存摺', '內容': '伸手請買壓長榮\n跌了就會做功課

In [ ]:
#轉成csv
df.reset_index(drop=True).to_csv("dcard.csv",encoding='utf-8-sig')

In [ ]:
# 讀取CSV檔案內容
import csv

with open('dcard.csv', newline='') as csvfile:
  rows = csv.reader(csvfile)
  for row in rows:
    print(row)

['', '主文/留言', '作者', '時間', '標題', '內容']
['0', '主文', '中國醫藥大學', '2022-02-27 15:18:51', '怎麼開始自立自強', '如題，跟爸爸媽媽說好，20歲開始自立自強，所有的學費跟生活費由我自己負擔。下個月要滿19歲了，所以我希望可以盡早規劃好金錢，以後才可以支付昂貴的學費給這間店\n\n先提一下目前的收入只有每個月生活費1萬，存款可以算0（買平板+駕訓班剛剛好花完了）。\n座標北港，因為實在太偏僻了，當家教也沒有人要，學校的校內打工這學期因為排班撞課所以沒有上，大一就沒有任何其他的收入。\n\n目前這學期有申請開通基富通的帳戶，想從每個月存3000元的基金開始（考慮的是安聯的台灣科技基金），並開始研究投資，然後等大二回台中後再去找打工開源\n\n這學期每個月金錢的安排如下\n3000買基金，2000車錢、5000其他\n\n如果是以20歲為分水嶺的話，我應該要大三才會開始自己付學費，但是我媽的脾氣也是很奇怪，我拿不準是大二下還是大三開始（現在就已經有成績不達標不付的機制了所以我不敢問），因此我希望要在明年的這個時候存到至少10萬（每個月3000+下半年的打工不知道行不行）\n\n存款預計需要支付一個學期的學費6萬+宿舍費2萬（出去租屋就是押金不知道多少）\n等自己開始自立自強之後（假設收入22K），1萬定存、6000租屋、6000生活，定存剛好一個學期清空一次去繳學費\n\n想問問看各位，現階段我還能做什麼讓我大二可以更輕鬆？然後對於未來金錢的規劃有何建議？我看到10萬覺得好害怕但是想想好像也不是不可能\n\n學貸的部分目前不是很想考慮，因為最後還是會變成媽媽付錢（大一考慮過，最後還是媽媽付）而且會變成一個我們兩個不愉快未爆彈，除非萬不得已不然暫時不考慮\n\n另外打工大概多久以前要開始找？一開始如果就去當便利商店或是手搖飲店員可以嗎？\n\n家教的部分，因為成績不是很好，如果以非直轄市第二志願的學歷（這樣形容感覺會被認出來笑死）+會考成績，去收國小國中的學生會不會找不到？我看家教社團的人都是大神我好怕\n（學測考超級爆炸爛也沒有指考，只能直接放棄放學測成績）\n\n謝謝看到這裡嗚嗚嗚嗚\nhttps://i.imgur.com/WUhYtOY.jpg']
['1', '留言', '國立臺灣大學',